In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import layers
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable warnings, info and errors 
import string
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import random
import time
from pathlib import Path

import re
from IPython import display
import tqdm

2022-01-01 17:42:44.178302: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcudart.so.11.0


In [2]:
cd /home/michael1017/work/data/comp3

/work/michael1017/data/comp3


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2022-01-01 17:42:45.550664: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcuda.so.1
2022-01-01 17:42:45.630519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-01-01 17:42:45.630557: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcudart.so.11.0
2022-01-01 17:42:45.637197: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcublas.so.11
2022-01-01 17:42:45.637252: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcublasLt.so.11
2022-01-01 17:42:45.639705: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcufft.

In [4]:
dictionary_path = './dictionary'
vocab = np.load(dictionary_path + '/vocab.npy')
print('there are {} vocabularies in total'.format(len(vocab)))

word2Id_dict = dict(np.load(dictionary_path + '/word2Id.npy'))
id2word_dict = dict(np.load(dictionary_path + '/id2Word.npy'))
print('Word to id mapping, for example: %s -> %s' % ('flower', word2Id_dict['flower']))
print('Id to word mapping, for example: %s -> %s' % ('1', id2word_dict['1']))
print('Tokens: <PAD>: %s; <RARE>: %s' % (word2Id_dict['<PAD>'], word2Id_dict['<RARE>']))

there are 5427 vocabularies in total
Word to id mapping, for example: flower -> 1
Id to word mapping, for example: 1 -> flower
Tokens: <PAD>: 5427; <RARE>: 5428


In [5]:
def sent2IdList(line, MAX_SEQ_LENGTH=20):
    MAX_SEQ_LIMIT = MAX_SEQ_LENGTH
    padding = 0
    
    # data preprocessing, remove all puntuation in the texts
    prep_line = re.sub('[%s]' % re.escape(string.punctuation), ' ', line.rstrip())
    prep_line = prep_line.replace('-', ' ')
    prep_line = prep_line.replace('-', ' ')
    prep_line = prep_line.replace('  ', ' ')
    prep_line = prep_line.replace('.', '')
    tokens = prep_line.split(' ')
    tokens = [
        tokens[i] for i in range(len(tokens))
        if tokens[i] != ' ' and tokens[i] != ''
    ]
    l = len(tokens)
    padding = MAX_SEQ_LIMIT - l
    
    # make sure length of each text is equal to MAX_SEQ_LENGTH, and replace the less common word with <RARE> token
    for i in range(padding):
        tokens.append('<PAD>')
    line = [
        word2Id_dict[tokens[k]]
        if tokens[k] in word2Id_dict else word2Id_dict['<RARE>']
        for k in range(len(tokens))
    ]

    return line

text = "the flower shown has yellow anther red pistil and bright red petals."
print(text)
print(sent2IdList(text))

the flower shown has yellow anther red pistil and bright red petals.
['9', '1', '82', '5', '11', '70', '20', '31', '3', '29', '20', '2', '5427', '5427', '5427', '5427', '5427', '5427', '5427', '5427']


In [6]:
data_path = './dataset'
df = pd.read_pickle(data_path + '/text2ImgData_3.pkl')
num_training_sample = len(df)
n_images_train = num_training_sample
print('There are %d image in training data' % (n_images_train))

There are 70504 image in training data


In [7]:
df.head(5)

,CaptionsPath,ImagePath
0,/home/michael1017/work/data/comp3/elmo_feature...,./102flowers/image_06734.jpg
1,/home/michael1017/work/data/comp3/elmo_feature...,./102flowers/image_06734.jpg
2,/home/michael1017/work/data/comp3/elmo_feature...,./102flowers/image_06734.jpg
3,/home/michael1017/work/data/comp3/elmo_feature...,./102flowers/image_06734.jpg
4,/home/michael1017/work/data/comp3/elmo_feature...,./102flowers/image_06734.jpg


In [8]:
a = tf.io.read_file("/home/michael1017/work/data/comp3" + "/elmo_features_3/" + "image_04150.jpg_2.npy")
a = tf.io.decode_raw(a, tf.float32)
npshape =1024
tensorshape = a.shape[0]
print(tensorshape)
tensorshape - npshape
a = a[tensorshape - npshape:] # npy value
# a = tf.reshape(
#     a, [20,1024], name=None
# )
print(a)
np.load("/home/michael1017/work/data/comp3" + "/elmo_features_3/" + "image_04150.jpg_2.npy")
#initialize the variable

1056
tf.Tensor([0.04699763 0.04215593 0.26952243 ... 0.01114284 0.22000797 0.46820766], shape=(1024,), dtype=float32)


array([0.04699763, 0.04215593, 0.26952243, ..., 0.01114284, 0.22000797,
       0.46820766], dtype=float32)

In [9]:
# in this competition, you have to generate image in size 64x64x3
IMG_H = 64
IMG_W = 64
IMAGE_CHANNEL = 3

# Load the numpy files
def map_func(img, cap):
    text_tensor = np.load(cap)
    return img, text_tensor



def training_data_generator(caption, image_path, wrong_img_path):
    # load in the image according to image path
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    short_side = tf.minimum(tf.shape(img)[0], tf.shape(img)[1])
    img = tf.image.resize_with_crop_or_pad(img, short_side, short_side)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.resize(img, [IMG_H, IMG_W])
    img = tf.cast(img, tf.float32)
    img = tf.clip_by_value(img, 0, 255)
    img = img/127.5 - 1.0
    
    wrong_img = tf.io.read_file(wrong_img_path)
    wrong_img = tf.image.decode_jpeg(wrong_img, channels=3)
    short_side = tf.minimum(tf.shape(wrong_img)[0], tf.shape(wrong_img)[1])
    wrong_img = tf.image.resize_with_crop_or_pad(wrong_img, short_side, short_side)
    wrong_img = tf.image.random_flip_left_right(wrong_img)
    wrong_img = tf.image.resize(wrong_img, [IMG_H, IMG_W])
    wrong_img = tf.cast(wrong_img, tf.float32)
    wrong_img = tf.clip_by_value(wrong_img, 0, 255)
    wrong_img = wrong_img/127.5 - 1.0
    
    a = tf.io.read_file(caption)
    a = tf.io.decode_raw(a, tf.float32)
    npshape = 1024
    tensorshape = 1056
    a = a[tensorshape - npshape:]
#     a = tf.reshape(
#         a, [20,1024], name=None
#     )
    
    return img, wrong_img, a

def dataset_generator(df, batch_size, data_generator):
    # load the training data into two NumPy arrays
    caption = df['CaptionsPath'].values
    image_path = df['ImagePath'].values
    wrong_image_path = tf.random.shuffle(image_path)
    
    # assume that each row of `features` corresponds to the same row as `labels`.
    assert caption.shape[0] == image_path.shape[0]
    
    dataset = tf.data.Dataset.from_tensor_slices((caption, image_path, wrong_image_path))
    dataset = dataset.map(data_generator, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(len(caption)).batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset




In [10]:
BATCH_SIZE = 64
dataset = dataset_generator(df, BATCH_SIZE, training_data_generator)

In [11]:
print(dataset)

<PrefetchDataset shapes: ((64, 64, 64, 3), (64, 64, 64, 3), (64, None)), types: (tf.float32, tf.float32, tf.float32)>


In [12]:
from tensorflow import keras
class Generator(tf.keras.Model):
    """
    Generate fake image based on given text(hidden representation) and noise z
    input: text and noise
    output: fake image with size 64*64*3
    """
    def __init__(self, hparas):
        super(Generator, self).__init__()
        self.hparas = hparas
        xh, xw, xc = self.hparas['IMAGE_SIZE']
        self.zh = xh // 4
        self.zw = xw // 4
        self.flt_1 = keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128)
#         self.d2 = tf.keras.layers.Dense(512)
#         self.d3 = tf.keras.layers.Dense(256)
#         self.d4 = tf.keras.layers.Dense(128)
        
        self.convT_1 = tf.keras.layers.Conv2DTranspose(filters = 512, kernel_size = 4, strides = 1, padding = "VALID",
                                                      kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_2 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.relu_3 = tf.keras.layers.LeakyReLU()
        # ------------ 8, 8, 256
        self.convT_4 = tf.keras.layers.Conv2DTranspose(filters = 256, kernel_size = 4, strides = 2, padding = "SAME",
                                                      kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_5 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.relu_6 = tf.keras.layers.LeakyReLU()
        # ------------ 16, 16, 128
        self.convT_7 = tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = 4, strides = 2, padding = "SAME",
                                                      kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_8 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.relu_9 = tf.keras.layers.LeakyReLU()
        # ------------ 32, 32, 64
        self.convT_10 = tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size = 4, strides = 2, padding = "SAME",
                                                       kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_11 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.relu_12 = tf.keras.layers.LeakyReLU()
        # ------------ 64, 64, 3
        self.convT_13 = tf.keras.layers.Conv2DTranspose(filters = 3, kernel_size = 4, strides = 2, padding = "SAME",
                                     activation = tf.keras.activations.tanh, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        
        
    def call(self, text, noise_z):
        text = self.flt_1(text)
        text = self.d1(text)
        text = tf.nn.leaky_relu(text)
#         text = self.d2(text)
#         text = tf.nn.leaky_relu(text)
#         text = self.d3(text)
#         text = tf.nn.leaky_relu(text)
#         text = self.d4(text)
#         text = tf.nn.leaky_relu(text)
        text = tf.expand_dims(text, axis=1)
        text = tf.expand_dims(text, axis=1)
        
        # concatenate input text and random noise
        text_concat = tf.concat([noise_z, text], axis=3)
    
        fake_image = self.convT_1(text_concat)
        fake_image = self.norm_2(fake_image)
        fake_image = self.relu_3(fake_image)
        fake_image = self.convT_4(fake_image)
        fake_image = self.norm_5(fake_image)
        fake_image = self.relu_6(fake_image)
        fake_image = self.convT_7(fake_image)
        fake_image = self.norm_8(fake_image)
        fake_image = self.relu_9(fake_image)
        fake_image = self.convT_10(fake_image)
        fake_image = self.norm_11(fake_image)
        fake_image = self.relu_12(fake_image)
        fake_image = self.convT_13(fake_image)
        
#         logits = tf.keras.layers.reshape(text_concat, [-1, 64, 64, 3])
#         output = tf.nn.tanh(logits)
        
        return fake_image

In [13]:
class Discriminator(tf.keras.Model):
    """
    Differentiate the real and fake image
    input: image and corresponding text
    output: labels, the real image should be 1, while the fake should be 0
    """
    def __init__(self, hparas):
        super(Discriminator, self).__init__()
        self.hparas = hparas
        self.flt_1 = keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128)
#         self.d2 = tf.keras.layers.Dense(512)
#         self.d3 = tf.keras.layers.Dense(256)
#         self.d4 = tf.keras.layers.Dense(128)
        self.d = tf.keras.layers.Dense(1)
        
        # ------ 32, 32, 64
        # ------ 32, 32, 64
        self.conv_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "SAME", input_shape = self.hparas['IMAGE_SIZE'],
                                            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.lrelu_2 = tf.keras.layers.LeakyReLU()
        # ------ 16, 16, 128
        self.conv_3 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides = 2, padding = "SAME",
                                            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_4 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.lrelu_5 = tf.keras.layers.LeakyReLU()
        # ------ 8, 8, 256
        self.conv_6 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 4, strides = 2, padding = "SAME",
                                            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_7 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.lrelu_8 = tf.keras.layers.LeakyReLU()
        # ------ 4, 4, 512
        self.conv_9 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 4, strides = 2, padding = "SAME",
                                            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        
        self.norm_10 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.lrelu_11 = tf.keras.layers.LeakyReLU()
    
        
        # ------ 4, 4, 512
        self.conv_12 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 1, strides = 1, padding = "SAME",
                                             kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))
        self.norm_13 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(stddev=0.02), epsilon=1e-5)
        self.lrelu_14 = tf.keras.layers.LeakyReLU()
        
        self.conv_15 = tf.keras.layers.Conv2D(filters = 1, kernel_size = 4, strides = 4, padding = "SAME")
        
        
    
    def call(self, img, text):
        text = self.flt_1(text)
        text = self.d1(text)
        text = tf.nn.leaky_relu(text)
#         text = self.d2(text)
#         text = tf.nn.leaky_relu(text)
#         text = self.d3(text)
#         text = tf.nn.leaky_relu(text)
#         text = self.d4(text)
#         text = tf.nn.leaky_relu(text)
        text_reduce = tf.expand_dims(text, axis=1)
        text_reduce = tf.expand_dims(text_reduce, axis=1)
        text_reduce = tf.tile(text_reduce, [1, 4, 4, 1])
        
        
#         img = self.flatten(img)
#         img = self.d_img(img)
#         img = tf.nn.leaky_relu(img)
        output = self.conv_1(img)
        output = self.lrelu_2(output)
        output = self.conv_3(output)
        output = self.norm_4(output)
        output = self.lrelu_5(output)
        output = self.conv_6(output)
        output = self.norm_7(output)
        output = self.lrelu_8(output)
        output = self.conv_9(output)
        output = self.norm_10(output)
        output = self.lrelu_11(output)
        
        
        text_concat = tf.concat([output, text_reduce], axis = 3)
        output = self.conv_12(text_concat)
        output = self.norm_13(output)
        output = self.lrelu_14(output)
        
        output = self.conv_15(output)
        
        
        logits = self.d(output)
#         output = tf.nn.sigmoid(logits)
        
        return logits

In [14]:
hparas = {
    'MAX_SEQ_LENGTH': 20,                     # maximum sequence length
    'EMBED_DIM': 256,                         # word embedding dimension
    'VOCAB_SIZE': len(word2Id_dict),          # size of dictionary of captions
    'RNN_HIDDEN_SIZE': 128,                   # number of RNN neurons
    'Z_DIM': 512,                             # random noise z dimension
    'DENSE_DIM': 1024,                         # number of neurons in dense layer
    'IMAGE_SIZE': [64, 64, 3],                # render image size
    'BATCH_SIZE': 64,
    'LR': 1e-4,
    'LR_DECAY': 0.5,
    'BETA_1': 0.5,
    'N_EPOCH': 1000,
    'N_SAMPLE': num_training_sample,          # size of training data
    'CHECKPOINTS_DIR': './checkpoints/demo',  # checkpoint path
    'PRINT_FREQ': 1                           # printing frequency of loss
}

In [15]:
# text_encoder = TextEncoder(hparas)
WG = Generator(hparas)
WD = Discriminator(hparas)

In [52]:
DC_LR = 1.0e-05
W_LR = 1.0e-05

In [53]:
optimizer_g = keras.optimizers.Adam(learning_rate=W_LR, beta_1=0.0, beta_2=0.9) # opt change to Adam 
optimizer_d = keras.optimizers.Adam(learning_rate=DC_LR, beta_1=0.0, beta_2=0.9)


@tf.function
def WGTrain(c1, text, fake_img, noise):
    epsilon = tf.random.normal([BATCH_SIZE, 1, 1, 1])
    beta = tf.random.normal([BATCH_SIZE, 1, 1, 1])

    with tf.GradientTape() as tpg:
        x0 = WG(text, noise, training = True)
        x1 = epsilon * c1 + (1 - epsilon) * beta * x0 +  (1 - epsilon) * (1 - beta) * x0
        
        dx0 = WD(x0, text, training = True)
        dc1 = WD(c1, text, training = True)
        
        with tf.GradientTape() as tape:  
            tape.watch(x1)
            dx1 = WD(x1, text, training = True) # gradient x1 on D(x1), so D(x1) should write in tape
            
        gra_dx1 = tape.gradient(dx1, x1)
        gra_norm =  tf.sqrt(tf.reduce_sum(tf.square(gra_dx1), axis = [1, 2, 3]))
        
        q = tf.reduce_mean(tf.square(gra_norm - 1.0)) * 10
        DX0 = tf.reduce_mean(dx0)
        DC1 = tf.reduce_mean(dc1)
        Lg = - DX0
        Ld = DX0 - DC1 + q
        
    gradient_g = tpg.gradient(Lg, WG.trainable_variables)
    optimizer_g.apply_gradients(zip(gradient_g, WG.trainable_variables))

    return Lg, Ld   



@tf.function
def WDTrain(c1, text, fake_img, noise):
    epsilon = tf.random.normal([BATCH_SIZE, 1, 1, 1])
#     beta = tf.random.normal([BATCH_SIZE, 1, 1, 1])

    with tf.GradientTape() as tpd:
        x0 = WG(text, noise, training = True)
        x1 = epsilon * c1 + (1 - epsilon)* x0 
        
        dx0 = WD(x0, text, training = True)
        dc1 = WD(c1, text, training = True)
        
        with tf.GradientTape() as tape:  
            tape.watch(x1)
            dx1 = WD(x1, text, training = True) # gradient x1 on D(x1), so D(x1) should write in tape
            
        gra_dx1 = tape.gradient(dx1, x1)
        gra_norm =  tf.sqrt(tf.reduce_sum(tf.square(gra_dx1), axis = [1, 2, 3]))
        
        q = tf.reduce_mean(tf.square(gra_norm - 1.0)) * 10
        DX0 = tf.reduce_mean(dx0)
        DC1 = tf.reduce_mean(dc1)
        Lg = - DX0
        Ld = DX0 - DC1 + q
        
    gradient_d = tpd.gradient(Ld, WD.trainable_variables)
    optimizer_d.apply_gradients(zip(gradient_d, WD.trainable_variables))

    return Lg, Ld   

In [54]:
WTrain = (
    WDTrain,
#     WDTrain,
    WGTrain
)

WCritic = len(WTrain)

In [55]:
@tf.function
def train_step(real_image, caption, fake_img, ctr):
    # random noise for generator
    noise = tf.random.normal(shape=[hparas['BATCH_SIZE'], 1, 1,hparas['Z_DIM']], mean=0.0, stddev=1.0)
#     text_embed, hidden = text_encoder(caption, hidden)
    lg, ld = WTrain[ctr](real_image, caption, fake_img, noise)
    return lg, ld

In [56]:
# one benefit of tf.train.Checkpoint() API is we can save everything seperately
checkpoint_dir = hparas['CHECKPOINTS_DIR']
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=optimizer_g,
                                 discriminator_optimizer=optimizer_d,
                                 generator=WG,
                                 discriminator=WD)

In [58]:
@tf.function
def test_step(caption, noise):
    text_embed = caption
    fake_image = WG(text_embed, noise)
    return fake_image

In [59]:
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

def imsave(images, size, path):
    # getting the pixel values between [0, 1] to save it
    return plt.imsave(path, merge(images, size)*0.5*0.99 + 0.5)

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

In [46]:
import tensorflow_hub as hub
elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["tokens"]

2022-01-02 05:29:44.912710: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:44.912743: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:44.912757: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:44.912763: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:45.152581: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:45.152622: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:45.152640: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:29:45.152647: W tensorflow/core/grappler/

In [47]:
def iDList2Sent(L):
    s_lst = []
    l_lst = []
    for sent in L:
        temp = []
        l = 0
        for i in sent:
            w = id2word_dict[i]
            if(w != "<PAD>"):
                temp.append(w)
                l += 1
            else:
                temp.append("")
        s_lst.append(temp)
        l_lst.append(l)
    return s_lst, l_lst

In [48]:
def sample_generator(caption, batch_size):
    s_lst, l_lst = iDList2Sent(caption)
    embedding = elmo(sequence_len = tf.constant(l_lst),
                     tokens = tf.constant(s_lst))["elmo"]
    embedding = tf.reduce_mean(embedding, 1)
    print(embedding)
    dataset = tf.data.Dataset.from_tensor_slices(embedding)
    dataset = dataset.batch(batch_size)
    return dataset

In [26]:
ni = int(np.ceil(np.sqrt(hparas['BATCH_SIZE'])))
sample_size = hparas['BATCH_SIZE']
sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, 1,1,hparas['Z_DIM'])).astype(np.float32)
sample_sentence = ["the flower shown has yellow anther red pistil and bright red petals."] * int(sample_size/ni) + \
                  ["this flower has petals that are yellow, white and purple and has dark lines"] * int(sample_size/ni) + \
                  ["the petals on this flower are white with a yellow center"] * int(sample_size/ni) + \
                  ["this flower has a lot of small round pink petals."] * int(sample_size/ni) + \
                  ["this flower is orange in color, and has petals that are ruffled and rounded."] * int(sample_size/ni) + \
                  ["the flower has yellow petals and the center of it is brown."] * int(sample_size/ni) + \
                  ["this flower has petals that are blue and white."] * int(sample_size/ni) +\
                  ["these white flowers have petals that start off white in color and end in a white towards the tips."] * int(sample_size/ni)

for i, sent in enumerate(sample_sentence):
    sample_sentence[i] = sent2IdList(sent)
#     print(i)
#     print(sent)

# sample_sentence = [sentense * 64]

# print(sample_sentence)
sample_sentence = sample_generator(sample_sentence, hparas['BATCH_SIZE'])

2022-01-01 17:42:48.680506: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:48.680538: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:48.680552: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:48.680557: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:48.841523: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1851] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hl

tf.Tensor(
[[ 0.06454559  0.12683736 -0.00523152 ... -0.00103128  0.03403852
   0.34140027]
 [ 0.06454559  0.12683736 -0.00523152 ... -0.00103128  0.03403852
   0.34140027]
 [ 0.06454559  0.12683736 -0.00523152 ... -0.00103128  0.03403852
   0.34140027]
 ...
 [ 0.10956667  0.3269495   0.29930332 ... -0.09462995  0.23867035
   0.40066648]
 [ 0.10956667  0.3269495   0.29930332 ... -0.09462995  0.23867035
   0.40066648]
 [ 0.10956667  0.3269495   0.29930332 ... -0.09462995  0.23867035
   0.40066648]], shape=(64, 1024), dtype=float32)


2022-01-01 17:42:50.959341: I tensorflow/compiler/jit/xla_compilation_cache.cc:342] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-01-01 17:42:50.974130: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:50.974150: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:50.974164: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:50.974169: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:50.975075: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 17:42:50.975093: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-01 

In [27]:
if not os.path.exists('samples/demo'):
    os.makedirs('samples/demo')

In [60]:
def train(dataset, epochs):
    # hidden state of RNN
    steps_per_epoch = int(hparas['N_SAMPLE']/hparas['BATCH_SIZE'])
    ctr = 0
    for epoch in range(hparas['N_EPOCH']):
        g_total_loss = 0
        d_total_loss = 0
        start = time.time()
        
        for image, fake_img, caption in dataset:
            g_loss, d_loss = train_step(image, caption, fake_img, ctr)
            ctr += 1
            g_total_loss += g_loss
            d_total_loss += d_loss
            if ctr == WCritic : ctr = 0
            
            
        time_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", time_tuple)
            
        print("Epoch {}, gen_loss: {:.4f}, disc_loss: {:.4f}".format(epoch+1,
                                                                     g_total_loss/steps_per_epoch,
                                                                     d_total_loss/steps_per_epoch))
        print('Time for epoch {} is {:.4f} sec'.format(epoch+1, time.time()-start))
        
        # save the model
        if (epoch + 1) % 50 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        
        # visualization
        if (epoch + 1) % hparas['PRINT_FREQ'] == 0:
            for caption in sample_sentence:
                fake_image = test_step(caption, sample_seed)
            try:
                save_images(fake_image, [ni, ni], 'samples/demo/train_{:02d}.jpg'.format(epoch))
            except:
                print("save failed")

In [61]:
train(dataset, hparas['N_EPOCH'])

2022-01-02 05:36:48.016185: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:36:48.016226: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:36:48.016240: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:36:48.016245: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:36:58.004399: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 10768 of 70504
2022-01-02 05:37:08.005257: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 21702 of 70504
2022-01-02 05:37:18.004628: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 32476 of 70504
2022

Epoch 1, gen_loss: -1331392.1250, disc_loss: -10.0740
Time for epoch 1 is 110.6955 sec


2022-01-02 05:38:38.712655: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.712686: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.712700: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.712705: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.748238: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.748270: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.748286: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:38:38.748292: W tensorflow/core/grappler/

Epoch 2, gen_loss: -1303858.2500, disc_loss: -4.2351
Time for epoch 2 is 100.9348 sec


2022-01-02 05:40:19.845303: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:40:19.845361: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:40:19.845374: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:40:19.845380: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 05:40:29.827618: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 10588 of 70504
2022-01-02 05:40:39.827653: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 21257 of 70504
2022-01-02 05:40:49.827836: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 31984 of 70504
2022

...

Epoch 332, gen_loss: nan, disc_loss: nan
Time for epoch 332 is 100.4379 sec


2022-01-02 14:51:07.436531: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:51:07.436585: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:51:07.436599: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:51:07.436604: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:51:17.421207: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 10818 of 70504
2022-01-02 14:51:27.421230: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 21823 of 70504
2022-01-02 14:51:37.421360: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 32782 of 70504
2022

Epoch 333, gen_loss: nan, disc_loss: nan
Time for epoch 333 is 99.1240 sec


2022-01-02 14:52:46.591747: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:52:46.591798: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:52:46.591812: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:52:46.591817: W tensorflow/core/grappler/optimizers/meta_optimizer.cc:153] TF_ENABLE_AUTO_MIXED_PRECISION has no effect.
2022-01-02 14:52:56.579835: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 10841 of 70504
2022-01-02 14:53:06.579726: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 21836 of 70504
2022-01-02 14:53:16.579535: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 32748 of 70504
2022

KeyboardInterrupt: 

In [57]:
checkpoint.restore(checkpoint_dir + '/ckpt-1')

In [ ]:
test_df = pd.read_pickle('./dataset/testData.pkl')

In [ ]:
test_df.head(5)

In [ ]:
# def iDList2Sent(L):
#     s_lst = []
#     l_lst = []
#     for sent in L:
#         temp = []
#         l = 0
#         for i in sent:
#             w = id2word_dict[i]
#             if(w != "<PAD>"):
#                 temp.append(w)
#                 l += 1
#             else:
#                 temp.append("")
#         s_lst.append(temp)
#         l_lst.append(l)
#     return s_lst, l_lst

In [ ]:
# def testing_data_generator(caption, index):
#     s_lst, l_lst = iDList2Sent([caption])
#     embedding = elmo(sequence_len = tf.constant(l_lst),
#                      tokens = tf.constant(s_lst))["elmo"]
#     embedding = tf.reduce_mean(embedding, 1)
# #     print(embedding)
# #     caption = tf.cast(caption, tf.float32)
#     return embedding, index

# def testing_dataset_generator(batch_size, data_generator):
#     data = pd.read_pickle('./dataset/testData.pkl')
#     captions = data['Captions'].values
#     caption = []
#     for i in range(len(captions)):
#         caption.append(captions[i])
        
#     s_lst, l_lst = iDList2Sent(caption)
#     embedding = elmo(sequence_len = tf.constant(l_lst),
#                  tokens = tf.constant(s_lst))["elmo"]
#     embedding = tf.reduce_mean(embedding, 1)
    
#     caption = np.asarray(caption)
#     caption = caption.astype(np.int)
#     index = data['ID'].values
#     index = np.asarray(index)
    
#     dataset = tf.data.Dataset.from_tensor_slices((caption, index))
# #     dataset = dataset.map(testing_data_generator, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     dataset = dataset.repeat().batch(batch_size)
    
#     return dataset

In [ ]:
# testing_dataset = testing_dataset_generator(hparas['BATCH_SIZE'], testing_data_generator)